## read data from files netatmo

In [6]:
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score
        
%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [7]:
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
data=pd.read_csv('dataCompletAvrilto18Juin.csv', sep=";", encoding = "ISO-8859-1", index_col=index_col)

data=data.iloc[0:14400,:]

In [8]:
data.shape

(14400, 6)

In [5]:
#data.to_csv('dataCompletTo20mai.csv') 

In [9]:
import pandas as pd
bear_family_df = pd.DataFrame(data)
bear_family_df.isna().sum()

Temperature    0
Humidity       0
CO2            0
Noise          0
Pressure       0
activity       0
dtype: int64

In [17]:
#data.to_csv('data.csv') 

## Missing data generation

In [18]:

#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

In [19]:
# Fix the seed ------------------------------------------------------
np.random.seed(0)

In [20]:
X = data.copy()
X = X.drop(["activity"], axis=1)
X = X.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X)
X= datascaler.transform(X)

In [22]:
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
df=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values


In [23]:
mask

array([[1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1.],
       [1., 0., 0., 1., 0.]])

## Time Series Imputation Techniques

In [24]:
### Forward Fill
# Impute airquality DataFrame with ffill method
import numpy as np
from sklearn.impute import KNNImputer
X1 = df.copy()
# add time "timpstame" as variable 
X1["time"]=X1.index
k=2
imputer = KNNImputer(n_neighbors=k)
KNNimputed = imputer.fit_transform(X1)
KNNimputed=KNNimputed[:,0:-1].copy()
KNNimputed= pd.DataFrame(data=KNNimputed, columns=df.columns)
KNNimputed.index = df.index

ffill_imputed = df.copy(deep=True)
ffill_imputed.fillna(method='ffill',inplace=True)
ffill_imputed.fillna(method='bfill',inplace=True)


bfill_imputed = df.copy(deep=True)
bfill_imputed.fillna(method='bfill',inplace=True)
bfill_imputed.fillna(method='ffill',inplace=True)


quadratic_imput = df.copy(deep=True)
quadratic_imput.interpolate(method='quadratic', inplace=True)
quadratic_imput.fillna(method='ffill',inplace=True)
quadratic_imput.fillna(method='bfill',inplace=True)


nearest_imput = df.copy(deep=True)
nearest_imput.interpolate(method='nearest', inplace=True)
nearest_imput.fillna(method='ffill',inplace=True)
nearest_imput.fillna(method='bfill',inplace=True)


linear_imput=df.copy(deep=True)
linear_imput.interpolate(method='linear', inplace=True)
linear_imput.fillna(method='ffill',inplace=True)
linear_imput.fillna(method='bfill',inplace=True)


poly_imput=df.copy(deep=True)
poly_imput= poly_imput.interpolate(method='polynomial', order=3)
poly_imput.fillna(method='ffill',inplace=True)
poly_imput.fillna(method='bfill',inplace=True)


# Save dataFrame Imputed

In [25]:
# Create a dictionary of interpolations
interpolations = {'Back_fill':bfill_imputed,
                  'Forward_fill':ffill_imputed,
                  'Linear_Interpolation': linear_imput,
                  'Quadratic_Interpolation':quadratic_imput,
                  "Nearest_imput":nearest_imput, 
                  "polynomial_interpolation":poly_imput,
                  'KNN': KNNimputed}

# for df_key in  interpolations:
#     print(df_key,type(interpolations[df_key]))
    
        

    

for df_key in  interpolations:
    if df_key == 'KNN' :
        Ximp = interpolations[df_key]
    else :
        Ximp = interpolations[df_key].values
        
    for i in range(5):
        print(df_key, "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
        
    print(df_key, " all ", RMSE(Ximp, X, mask))
    


    

Back_fill   Temperature 0.057794405024306994
Back_fill   Humidity 0.04517357034102688
Back_fill   CO2 0.16530767405289135
Back_fill   Noise 0.650338799598682
Back_fill   Pressure 0.014407929937218376
Back_fill  all  0.30314973974593346
Forward_fill   Temperature 0.0567944637893074
Forward_fill   Humidity 0.04614515685668797
Forward_fill   CO2 0.16566224255446002
Forward_fill   Noise 0.6430073403370705
Forward_fill   Pressure 0.014378275796837939
Forward_fill  all  0.30000940274939325
Linear_Interpolation   Temperature 0.03105969547966727
Linear_Interpolation   Humidity 0.028688778325488717
Linear_Interpolation   CO2 0.10276781307047578
Linear_Interpolation   Noise 0.49417355092200527
Linear_Interpolation   Pressure 0.009007733792601464
Linear_Interpolation  all  0.22748299105764333
Quadratic_Interpolation   Temperature 0.03843520494285436
Quadratic_Interpolation   Humidity 0.03423625180167006
Quadratic_Interpolation   CO2 0.1459111144206467
Quadratic_Interpolation   Noise 1.48136331687

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [114]:
p

NameError: name 'p' is not defined

In [145]:
#quadratic_imput
df.columns


Index(['Temperature', 'Humidity', 'CO2', 'Noise', 'Pressure'], dtype='object')

In [28]:
X_kalman=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1")
X_kalmanArimaImputed=pd.read_csv('./dataImputed/kalmanArima_TRUE_MCAR40.csv', sep=",", encoding = "ISO-8859-1")
X_kalmanStructTsImputed=pd.read_csv('./dataImputed/kalmanStructTS_TRUE_MCAR40.csv', sep=",", encoding = "ISO-8859-1")

In [29]:
X_kalmanArimaImputed=X_kalmanArimaImputed.set_index('time')
X_kalmanStructTsImputed=X_kalmanStructTsImputed.set_index('time')

In [ ]:
X_kalmanStructTsImputed

In [30]:
Ximp= X_kalmanArimaImputed.values

model_kalman ="kalmanArima_TRUE_MCAR5"
#print(model_kalman)
for i in range(5):
    print(model_kalman, "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
print(model_kalman, " all ", RMSE(Ximp, X, mask))
   

kalmanArima_TRUE_MCAR5   Temperature 0.03105969547972639
kalmanArima_TRUE_MCAR5   Humidity 0.028874161294569902
kalmanArima_TRUE_MCAR5   CO2 0.1027678130704758
kalmanArima_TRUE_MCAR5   Noise 0.49453465273458225
kalmanArima_TRUE_MCAR5   Pressure 0.009036438228566712
kalmanArima_TRUE_MCAR5  all  0.22764614025624014


In [16]:
df.head(10)

,Temperature,Humidity,CO2,Noise,Pressure
time,,,,,
1617235589,NaN,0.369908,-0.071631,-0.531945,0.195873
1617235891,0.293575,NaN,NaN,NaN,0.195873
1617236194,0.293575,0.369908,NaN,-0.531945,NaN
1617236497,0.293575,NaN,NaN,NaN,0.195873
1617236799,NaN,0.369908,NaN,NaN,0.195873
1617237102,0.251304,0.239908,-0.272994,-0.531945,0.195873
1617237405,0.251304,0.239908,NaN,-0.531945,NaN
1617237707,0.251304,0.239908,NaN,NaN,0.195873
1617238010,NaN,NaN,-0.223772,-0.531945,NaN


In [151]:
# Create a dictionary of interpolations
interpolations = {'Back-fill':bfill_imputed, 'Forward-fill':ffill_imputed,
                  'Linear Interpolation': linear_imput, 'Quadratic Interpolation': quadratic_imput,"nearest_imput":nearest_imput, "polynomial interpolation":poly_imput, 'KNN': KNNimputed}
#mask = R_mcar

n_neighbors=20

# Loop over axes and interpolations
print("KNN K= ", n_neighbors)
for df_key in  interpolations:
    if df_key == 'KNN' :
        Ximp = interpolations[df_key]
    else :
        Ximp = interpolations[df_key].values
        
    for i in range(5):
        print(df_key, "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
        
    print(df_key, " all ", RMSE(Ximp, X, mask))  


KNN K=  20
Back-fill   Temperature 0.023779007725629266
Back-fill   Humidity 0.028236604870544846
Back-fill   CO2 0.11158811684864746
Back-fill   Noise 0.42607945846866513
Back-fill   Pressure 0.012309376731082636
Back-fill  all  0.19437012695215714
Forward-fill   Temperature 0.02424206318450242
Forward-fill   Humidity 0.03374919760850409
Forward-fill   CO2 0.12573825182153567
Forward-fill   Noise 0.4876529728991461
Forward-fill   Pressure 0.012861433045138692
Forward-fill  all  0.22219328378225472
Linear Interpolation   Temperature 0.017268022613511704
Linear Interpolation   Humidity 0.02202424240047725
Linear Interpolation   CO2 0.07521173994273728
Linear Interpolation   Noise 0.34330267242793633
Linear Interpolation   Pressure 0.009244767261754673
Linear Interpolation  all  0.15496362862629107
Quadratic Interpolation   Temperature 0.021043107204542305
Quadratic Interpolation   Humidity 0.03150192524658523
Quadratic Interpolation   CO2 0.09403220969187376
Quadratic Interpolation   No

In [ ]:

datascaler.inverse_transform(quadratic_imput)

In [29]:
from sklearn.preprocessing import StandardScaler
X1 = df.copy()
X1["time"]=X1.index
scaler_time = StandardScaler()
indexscaler=scaler_time.fit(X1)
X2 = indexscaler.transform(X1)

In [33]:
X2[:,5]

array([-1.73756519, -1.73741153, -1.73725737, ...,  1.73486047,
        1.73501413,  1.73516779])

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
X1 = df.copy()
X1["time"]=X1.index
print(X1.columns)
for k in range(1,10):
    imputer = KNNImputer(n_neighbors=k)
    KNNimputed = imputer.fit_transform(X1)
    Ximp= KNNimputed    
    Ximp = Ximp[:,0:-1]
    print("k = ",k)
    for i in range(5):
        print("KNN", "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
    print("KNN", " all ", RMSE(Ximp, X,mask))

In [54]:
imputer = MissForest(max_iter=12, n_jobs=-1)
X_imputed = imputer.fit_transform(df)
df1 = pd.DataFrame(X_imputed)
df1.head()

NameError: name 'MissForest' is not defined

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# Setting the random_state argument for reproducibility
imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(df)
df_imputed = pd.DataFrame(imputed, columns=df.columns)
df_key="IterativeImputer"
round(df_imputed, 2)


In [ ]:
Ximp= y_hat.values
print("k = ",k)
for i in range(5):
    print(df_key, "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
print(df_key, " all ", RMSE(Ximp, X, mask))    

In [ ]:
y.shape

In [ ]:
data.shape

In [ ]:
from sktime.transformations.series.impute import Imputer
from sktime.datasets import load_airline
y = df.copy()
print(y.shape)
import datetime

# current date and time

i=0
a=[]

for t in y.index:
    a.append(datetime.datetime.fromtimestamp(t))
    
y.index=a
print(y.shape)
y = y.sort_index()
print(y.shape)
y=y.resample("300S").mean()
print(y.shape)
transformer = Imputer(method="drift")
y_hat = transformer.fit_transform(y)

In [ ]:
a=[]
for t in data.index:
    a.append(datetime.datetime.fromtimestamp(t))
    
data.index=a

In [ ]:
y

In [ ]:
db

In [ ]:
df.index

In [ ]:
import datetime

# current date and time

i=0
a=[]

for t in y.index:
    a.append(datetime.datetime.fromtimestamp(t))     


In [ ]:
y=y.resample("5T").mean()

In [ ]:
plt.plot(y.index)